In [2]:
import numpy as np
import tensorflow as tf


import sys

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
train_data = None
dev_data = None
test_data = None
with open('data/train_data.bin', 'rb') as file:
    train_data = np.load(file)
with open('data/dev_data.bin', 'rb') as file:
    dev_data = np.load(file)
with open('data/test_data.bin', 'rb') as file:
    test_data = np.load(file)
if(train_data is None or dev_data is None or test_data is None):
    print("Could not load all data")

In [4]:
flipped_train_data = np.asarray([np.column_stack((game[:,1],game[:,0])) for game in train_data])
flipped_dev_data = np.asarray([np.column_stack((game[:,1],game[:,0])) for game in dev_data])
flipped_test_data = np.asarray([np.column_stack((game[:,1],game[:,0])) for game in test_data])

In [5]:
# Create the model
feat = None
def nn_model_fn(features, labels, mode):
    
    hidden = tf.layers.dense(features['x'], 10, tf.nn.relu)
    logits = tf.layers.dense(hidden, 1, tf.nn.softmax)

    predictions = {
        # Generate predictions (for PREDICT and EVAL mode)
        "classes": tf.argmax(input=logits, axis=1),
        # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
        # `logging_hook`.
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    # Calculate Loss (for both TRAIN and EVAL modes)
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Configure the Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(loss=loss, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = { "accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"]) }
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

In [6]:
# Create the Estimator
league_classifier = tf.estimator.Estimator(model_fn=nn_model_fn, model_dir="nn_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'nn_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001A92DA73F60>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [7]:
# Set up logging for predictions
# Log the values in the "Softmax" tensor with label "probabilities"
tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=200)

In [21]:
# Train the model
'''
x={"x": flipped_train_data},
y=np.zeros(len(train_data), dtype=np.int32),

x={"x": np.concatenate([train_data,flipped_train_data])},
y=np.concatenate([np.ones(len(train_data), dtype=np.int32), np.zeros(len(train_data), dtype=np.int32)]),
'''
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": np.concatenate([train_data,flipped_train_data])},
        y=np.concatenate([np.ones(len(train_data), dtype=np.int32), np.zeros(len(train_data), dtype=np.int32)]),
        batch_size=100,
        num_epochs=None,
        shuffle=True)
league_classifier.train(
      input_fn=train_input_fn,
      steps=600,
      hooks=[logging_hook])

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from nn_model\model.ckpt-1


KeyboardInterrupt: 

In [109]:
# Evaluate the model and print results
# x={"x": np.concatenate([dev_data,flipped_dev_data])},
# y=np.concatenate([np.ones(len(dev_data), dtype = np.int32), np.zeros(len(dev_data), dtype = np.int32)])
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": dev_data},
  y=np.ones(len(dev_data), dtype = np.int32),
  num_epochs=1,
  shuffle=False)
eval_results = league_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-04-20:34:01
INFO:tensorflow:Restoring parameters from nn_model\model.ckpt-200
INFO:tensorflow:Finished evaluation at 2018-03-04-20:34:01
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.0, global_step = 200, loss = 4.934474
{'accuracy': 0.0, 'loss': 4.934474, 'global_step': 200}
